# Testes de treinamento incremental

A ideia deste notebook é verificar se treinar os algoritmos de MAB durante as interações de teste geram alguma melhora para eles, comparando-os com os resultados de um algoritmo estático (ALS).

In [1]:
import pandas as pd
from mab2rec import BanditRecommender, LearningPolicy
from mab2rec.pipeline import train
from sklearn.preprocessing import LabelEncoder
import implicit
from scipy.sparse import csr_matrix
from implicit.nearest_neighbours import bm25_weight

train_data = "../data/ml100k/data_train.csv"
test_data = "../data/ml100k/data_test.csv"

In [2]:
def initial_train(df, num_users, num_items):
    FACTORS = 10
    K1 = 100
    B = 0.8

    # Cria a matriz esparsa
    sparse_matrix = csr_matrix((df['response'], (df['user_id'], df['item_id'])), shape=(num_users, num_items))
    updated_sparse_matrix = bm25_weight(sparse_matrix, K1=K1, B=B)

    print('Treinando o modelo ALS')
    ALS_model = implicit.als.AlternatingLeastSquares(factors=FACTORS, random_state=1)
    ALS_model.fit(updated_sparse_matrix)

    user_features_list = []

    for user_id in df['user_id'].unique():
        user_factors = ALS_model.user_factors[user_id]
        user_features_list.append([user_id] + list(user_factors))

    df_user_features = pd.DataFrame(user_features_list, columns=['user_id'] + [f'u{i}' for i in range(FACTORS)])

    print('Treinando o modelo LinUCB')
    linUCB_model = BanditRecommender(learning_policy=LearningPolicy.LinUCB(), top_k=10)
    train(linUCB_model, data=df, user_features=df_user_features)

    print('Treinando o modelo LinGreedy')
    linGreedy_model = BanditRecommender(learning_policy=LearningPolicy.LinGreedy(), top_k=10)
    train(linGreedy_model, data=df, user_features=df_user_features)

    return ALS_model, linUCB_model, linGreedy_model, df_user_features, sparse_matrix

In [3]:
def test_ALS(ALS_model, sparse_matrix, df_test):
    print('Testing ALS')
    hits = 0
    for i, interaction in df_test.iterrows():
        ids_recs, _ = ALS_model.recommend(userid=interaction['user_id'], user_items=sparse_matrix[interaction['user_id']], N=10)
        if interaction['item_id'] in ids_recs:
            hits += 1

    print('\nALS RESULTS:')
    print(f'Hits: {hits}')
    print(f'Total: {len(df_test)}')
    print(f'HR: {hits/len(df_test)}\n')

In [4]:
def test_non_incremental(mab_algo, algo_name, user_features, df_test):
    print(f'Testing {algo_name}')
    hits = 0

    contexts = df_test.merge(user_features, on='user_id').drop(columns=['user_id', 'item_id', 'response']).values

    recomendations = mab_algo.recommend(contexts)

    df_test = df_test.reset_index(drop=True)

    hits = 0
    for i, interaction in df_test.iterrows():
        if interaction['item_id'] in recomendations[i]:
            hits += 1

    print(f'\n{algo_name} RESULTS:')
    print(f'Hits: {hits}')
    print(f'Total: {len(df_test)}')
    print(f'HR: {hits/len(df_test)}\n')

In [5]:
def test_incremental(mab_algo, algo_name, user_features, df_test, df_test_for_evaluation, batch_size):
    print(f'Testing {algo_name}')
    hits = 0

    for i in range(0, len(df_test), batch_size):
        # Fazendo recomendações para teste
        df_batch_test = df_test_for_evaluation.loc[i:i+batch_size-1]
        contexts = df_batch_test.merge(user_features, on='user_id').drop(columns=['user_id', 'item_id', 'response']).values

        if len(contexts) > 0:
            recomendations = mab_algo.recommend(contexts)
            if isinstance(recomendations, list) and isinstance(recomendations[0], int):
                recomendations = [recomendations]

            df_batch_test = df_batch_test.reset_index(drop=True)

            for j, interaction in df_batch_test.iterrows():
                if interaction['item_id'] in recomendations[j]:
                    hits += 1
        
        # Treinando com o batch
        df_batch_train = df_test.loc[i:i+batch_size-1]
        contexts = df_batch_train.merge(user_features, on='user_id').drop(columns=['user_id', 'item_id', 'response']).values

        mab_algo.partial_fit(df_batch_train['item_id'], df_batch_train['response'], contexts)

    print(f'\n{algo_name} RESULTS:')
    print(f'Hits: {hits}')
    print(f'Total: {len(df_test_for_evaluation)}')
    print(f'HR: {hits/len(df_test_for_evaluation)}\n')

In [6]:
def test(train_size, batch_size):
    df_train = pd.read_csv(train_data)
    df_test = pd.read_csv(test_data)

    df_full = pd.concat([df_train, df_test])

    df_full['user_id'] = LabelEncoder().fit_transform(df_full['user_id'])
    df_full['item_id'] = LabelEncoder().fit_transform(df_full['item_id'])

    num_users = df_full['user_id'].nunique()
    num_items = df_full['item_id'].nunique()

    split_index = int(len(df_full) * train_size)
    df_train = df_full[:split_index]
    df_test = df_full[split_index:]

    df_test = df_test[(df_test['user_id'].isin(df_train['user_id'])) & (df_test['item_id'].isin(df_train['item_id']))]
    df_test = df_test.reset_index(drop=True)
    df_test_for_evaluation = df_test[df_test['response'] == 1]

    ALS_model, linUCB_model, linGreedy_model, df_user_features, sparse_matrix = initial_train(df_train, num_users, num_items)

    test_ALS(ALS_model, sparse_matrix, df_test_for_evaluation)

    print('\nTesting non-incremental\n')
    test_non_incremental(linUCB_model, 'LinUCB', df_user_features, df_test_for_evaluation)
    test_non_incremental(linGreedy_model, 'LinGreedy', df_user_features, df_test_for_evaluation)

    print('\nTesting incremental\n')
    test_incremental(linUCB_model, 'LinUCB', df_user_features, df_test, df_test_for_evaluation, batch_size)
    test_incremental(linGreedy_model, 'LinGreedy', df_user_features, df_test, df_test_for_evaluation, batch_size)

In [7]:
test(0.9, 10)

Treinando o modelo ALS


c:\Users\gregf\anaconda3\envs\RLRS-rllib\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0009992122650146484 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

Treinando o modelo LinUCB
Treinando o modelo LinGreedy
Testing ALS

ALS RESULTS:
Hits: 82
Total: 623
HR: 0.13162118780096307


Testing non-incremental

Testing LinUCB

LinUCB RESULTS:
Hits: 47
Total: 623
HR: 0.0754414125200642

Testing LinGreedy

LinGreedy RESULTS:
Hits: 50
Total: 623
HR: 0.08025682182985554


Testing incremental

Testing LinUCB

LinUCB RESULTS:
Hits: 43
Total: 623
HR: 0.06902086677367576

Testing LinGreedy

LinGreedy RESULTS:
Hits: 51
Total: 623
HR: 0.08186195826645265



In [15]:
test(0.9, 250)

Treinando o modelo ALS


c:\Users\gregf\anaconda3\envs\RLRS-rllib\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

Treinando o modelo LinUCB
Treinando o modelo LinGreedy
Testing ALS

ALS RESULTS:
Hits: 82
Total: 623
HR: 0.13162118780096307


Testing non-incremental

Testing LinUCB

LinUCB RESULTS:
Hits: 47
Total: 623
HR: 0.0754414125200642

Testing LinGreedy

LinGreedy RESULTS:
Hits: 50
Total: 623
HR: 0.08025682182985554


Testing incremental

Testing LinUCB

LinUCB RESULTS:
Hits: 42
Total: 623
HR: 0.06741573033707865

Testing LinGreedy

LinGreedy RESULTS:
Hits: 46
Total: 623
HR: 0.0738362760834671



In [9]:
test(0.5, 10)

Treinando o modelo ALS


c:\Users\gregf\anaconda3\envs\RLRS-rllib\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

Treinando o modelo LinUCB
Treinando o modelo LinGreedy
Testing ALS

ALS RESULTS:
Hits: 955
Total: 4726
HR: 0.20207363520947946


Testing non-incremental

Testing LinUCB

LinUCB RESULTS:
Hits: 328
Total: 4726
HR: 0.06940330088870081

Testing LinGreedy

LinGreedy RESULTS:
Hits: 378
Total: 4726
HR: 0.0799830723656369


Testing incremental

Testing LinUCB

LinUCB RESULTS:
Hits: 395
Total: 4726
HR: 0.08358019466779518

Testing LinGreedy

LinGreedy RESULTS:
Hits: 479
Total: 4726
HR: 0.10135421074904782



In [10]:
test(0.5, 250)

Treinando o modelo ALS


c:\Users\gregf\anaconda3\envs\RLRS-rllib\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

Treinando o modelo LinUCB
Treinando o modelo LinGreedy
Testing ALS

ALS RESULTS:
Hits: 955
Total: 4726
HR: 0.20207363520947946


Testing non-incremental

Testing LinUCB

LinUCB RESULTS:
Hits: 328
Total: 4726
HR: 0.06940330088870081

Testing LinGreedy

LinGreedy RESULTS:
Hits: 378
Total: 4726
HR: 0.0799830723656369


Testing incremental

Testing LinUCB

LinUCB RESULTS:
Hits: 425
Total: 4726
HR: 0.08992805755395683

Testing LinGreedy

LinGreedy RESULTS:
Hits: 449
Total: 4726
HR: 0.09500634786288616



In [11]:
test(0.1, 10)

Treinando o modelo ALS


c:\Users\gregf\anaconda3\envs\RLRS-rllib\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

Treinando o modelo LinUCB
Treinando o modelo LinGreedy
Testing ALS

ALS RESULTS:
Hits: 824
Total: 8509
HR: 0.09683864143847691


Testing non-incremental

Testing LinUCB

LinUCB RESULTS:
Hits: 389
Total: 8509
HR: 0.045716300387824656

Testing LinGreedy

LinGreedy RESULTS:
Hits: 580
Total: 8509
HR: 0.06816312140086966


Testing incremental

Testing LinUCB

LinUCB RESULTS:
Hits: 516
Total: 8509
HR: 0.060641673522153015

Testing LinGreedy

LinGreedy RESULTS:
Hits: 643
Total: 8509
HR: 0.07556704665648137



In [12]:
test(0.1, 250)

Treinando o modelo ALS


c:\Users\gregf\anaconda3\envs\RLRS-rllib\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

Treinando o modelo LinUCB
Treinando o modelo LinGreedy
Testing ALS

ALS RESULTS:
Hits: 824
Total: 8509
HR: 0.09683864143847691


Testing non-incremental

Testing LinUCB

LinUCB RESULTS:
Hits: 389
Total: 8509
HR: 0.045716300387824656

Testing LinGreedy

LinGreedy RESULTS:
Hits: 580
Total: 8509
HR: 0.06816312140086966


Testing incremental

Testing LinUCB

LinUCB RESULTS:
Hits: 520
Total: 8509
HR: 0.06111176401457281

Testing LinGreedy

LinGreedy RESULTS:
Hits: 655
Total: 8509
HR: 0.07697731813374074



In [13]:
test(0.05, 10)

Treinando o modelo ALS


c:\Users\gregf\anaconda3\envs\RLRS-rllib\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0005083084106445312 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

Treinando o modelo LinUCB
Treinando o modelo LinGreedy
Testing ALS

ALS RESULTS:
Hits: 749
Total: 8242
HR: 0.0908760009706382


Testing non-incremental

Testing LinUCB

LinUCB RESULTS:
Hits: 556
Total: 8242
HR: 0.06745935452560058

Testing LinGreedy

LinGreedy RESULTS:
Hits: 556
Total: 8242
HR: 0.06745935452560058


Testing incremental

Testing LinUCB

LinUCB RESULTS:
Hits: 484
Total: 8242
HR: 0.05872361077408396

Testing LinGreedy

LinGreedy RESULTS:
Hits: 602
Total: 8242
HR: 0.07304052414462509



In [14]:
test(0.05, 250)

Treinando o modelo ALS


c:\Users\gregf\anaconda3\envs\RLRS-rllib\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

Treinando o modelo LinUCB
Treinando o modelo LinGreedy
Testing ALS

ALS RESULTS:
Hits: 749
Total: 8242
HR: 0.0908760009706382


Testing non-incremental

Testing LinUCB

LinUCB RESULTS:
Hits: 556
Total: 8242
HR: 0.06745935452560058

Testing LinGreedy

LinGreedy RESULTS:
Hits: 556
Total: 8242
HR: 0.06745935452560058


Testing incremental

Testing LinUCB

LinUCB RESULTS:
Hits: 515
Total: 8242
HR: 0.062484833778209174

Testing LinGreedy

LinGreedy RESULTS:
Hits: 659
Total: 8242
HR: 0.07995632128124242

